# Designing Specifications

Objective

* 了解非确定性规范，并能识别和评估非确定性规范
* Understand declarative vs. operational specs, and be able to write declarative specs
* 了解前提条件、后置条件和规范的强度，并能比较规范的强度
* 能够编写连贯、有用、强度适当的规范


## Introduction
In this reading we’ll look at different specs for similar behaviors, and talk about the tradeoffs between them. We’ll look at three dimensions for comparing specs:

* How deterministic it is. Does the spec define only a single possible output for a given input, or does it permit a set of legal outputs?

* How declarative it is. Does the spec just characterize what the output should be, or does it explicitly say how to compute the output?

* How strong it is. Does the spec have a small set of legal implementations, or a large set?

* Not all specifications we might choose for a module are equally useful, and we’ll explore what makes some specifications better than others.

## Deterministic vs. underdetermined specs
```TS
function findFirst(arr: Array<number>, val: number): number {
    for (let i = 0; i < arr.length; i++) {
        if (arr[i] === val) return i;
    }
    return arr.length;
}

function findLast(arr: Array<number>, val: number): number {
    for (let i = arr.length - 1 ; i >= 0; i--) {
        if (arr[i] === val) return i;
    }
    return -1;
}
```

下标 First 和 Last 并不是真正的 TypeScript 语法。为了便于讨论，我们在此使用它们来区分两种实现。在实际代码中，这两种实现都应该是名为 find 的 TypeScript 方法。

因为我们也会讨论 find 的多种规范，所以我们会用一个上标来标识每种规范，比如 ExactlyOne：

```
function findExactlyOne(arr: Array<number>, val: number): number
requires: val occurs exactly once in arr
effects: returns index i such that arr[i] = val
```

findExactlyOne 规范是deterministic：当出现满足先决条件的状态时，结果是完全确定的。只有一个返回值和一个最终状态是可能的。没有任何有效输入会有一个以上的有效输出。

findFirst 和 findLast 的实现都满足该规范的要求，因此，如果这是客户所依赖的规范，那么这两个实现是可以相互替代的。

下面是一个稍有不同的规范：

```
function findOneOrMore,AnyIndex(arr: Array<number>, val: number): number
requires: val occurs in arr
effects: returns index i such that arr[i] = val
```
该规范不是deterministic。它没有说明如果 val 出现多次，会返回哪个索引。它只是说，如果按照返回值给出的索引查找条目，就能找到 val。在某些输入上，该规范允许多个有效输出。

尽管这种规范不是deterministic，但我们不会把它称为通常意义上的nondeterministic。Nondeterministic代码有时表现为一种方式，有时又表现为另一种方式，即使在同一程序中以相同的输入调用也是如此。例如，当代码的行为取决于随机数，或取决于并发进程的时序时，就会出现这种情况。但是，非确定性规范并不一定需要非确定性实现。完全确定的实现也能满足它。

为了避免混淆，我们把not deterministic spec 称为underdetermined spec。

underdetermined findOneOrMore,AnyIndex spec同样可以通过 findFirst 和 findLast 来实现，它们各自以自己的（fully deterministic）方式解决underdeterminedness 问题。如果 val 出现不止一次，该规范的客户端就不能依赖于返回哪个索引。非确定性实现也能满足该规范的要求，例如，可以通过掷硬币来决定从数组的开头还是结尾开始搜索。但在我们会遇到的几乎所有情况下，规范中的非确定性都提供了一种由实现者在实现时做出的选择。未完全确定的规范通常由完全确定的实现来实现。
findFirst 和 findLast 都能满足这种查找一个或多个、任意索引的欠确定性规范，它们各自以自己的（完全确定的）方式解决欠确定性问题。如果 val 出现不止一次，该规范的客户端就不能依赖于返回哪个索引。非确定性实现也能满足该规范的要求，例如，可以通过掷硬币来决定从数组的开头还是结尾开始搜索。但在我们会遇到的几乎所有情况下，规范中的非确定性都提供了一种由实现者在实现时做出的选择。未完全确定的规范通常由完全确定的实现来实现。